In [107]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
import numpy as np
import csv

In [108]:
Result = pd.read_csv('extract/raw-data.csv')
columns=['Unnamed: 0']
Result.drop(columns, inplace=True, axis=1)
Result

,Rank,Bib,FIS code,Athlete,Year,Nation,Time,Diff. Time,FIS Points,Event,Event ID,Place,Date,Result Type,Season
0,1.0,31.0,3295157,SCARDONI Lucia,1991.0,ITA,3:03.01,3:03.01,0.00,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
1,2.0,12.0,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,3:04.16,+1.15,7.54,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
2,3.0,23.0,3295193,LAURENT Greta,1992.0,ITA,3:05.15,+2.14,14.03,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
3,4.0,4.0,3535304,CALDWELL Sophie,1990.0,USA,3:05.62,+2.61,17.11,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
4,5.0,27.0,3426112,STENSETH Ane Appelkvist,1995.0,NOR,3:05.67,+2.66,17.44,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6731,47.0,39.0,3530772,LUSTGARTEN Benjamin,1992.0,USA,2:21:25.4,+13:20.1,145.75,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6732,48.0,52.0,3181007,VUORELA Markus,1996.0,FIN,2:22:37.9,+14:32.6,158.96,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6733,49.0,56.0,3120063,SHANG Jincai,1993.0,CHN,2:23:13.9,+15:08.6,165.52,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6734,50.0,38.0,3530711,MARTIN Adam,1994.0,USA,2:23:15.4,+15:10.1,165.79,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020


In [110]:
Result.replace(r'^\s*$', np.nan, regex=True, inplace=True)
Result['Rank']=Result['Rank'].fillna(method='ffill')
Result.info()
Result


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6736 entries, 0 to 6735
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         6736 non-null   float64
 1   Bib          6385 non-null   float64
 2   FIS code     6736 non-null   int64  
 3   Athlete      5885 non-null   object 
 4   Year         6481 non-null   float64
 5   Nation       6140 non-null   object 
 6   Time         4366 non-null   object 
 7   Diff. Time   4555 non-null   object 
 8   FIS Points   4015 non-null   float64
 9   Event        6736 non-null   object 
 10  Event ID     6736 non-null   int64  
 11  Place        6736 non-null   object 
 12  Date         6736 non-null   object 
 13  Result Type  6736 non-null   object 
 14  Season       6736 non-null   int64  
dtypes: float64(4), int64(3), object(8)
memory usage: 789.5+ KB


,Rank,Bib,FIS code,Athlete,Year,Nation,Time,Diff. Time,FIS Points,Event,Event ID,Place,Date,Result Type,Season
0,1.0,31.0,3295157,SCARDONI Lucia,1991.0,ITA,3:03.01,3:03.01,0.00,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
1,2.0,12.0,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,3:04.16,+1.15,7.54,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
2,3.0,23.0,3295193,LAURENT Greta,1992.0,ITA,3:05.15,+2.14,14.03,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
3,4.0,4.0,3535304,CALDWELL Sophie,1990.0,USA,3:05.62,+2.61,17.11,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
4,5.0,27.0,3426112,STENSETH Ane Appelkvist,1995.0,NOR,3:05.67,+2.66,17.44,Women's SP C Qual,4331,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6731,47.0,39.0,3530772,LUSTGARTEN Benjamin,1992.0,USA,2:21:25.4,+13:20.1,145.75,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6732,48.0,52.0,3181007,VUORELA Markus,1996.0,FIN,2:22:37.9,+14:32.6,158.96,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6733,49.0,56.0,3120063,SHANG Jincai,1993.0,CHN,2:23:13.9,+15:08.6,165.52,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020
6734,50.0,38.0,3530711,MARTIN Adam,1994.0,USA,2:23:15.4,+15:10.1,165.79,Men's 50 km C Mst,4434,Oslo (NOR),"Mar 08, 2020",World Cup,2020


In [111]:
Result['Rank'] = Result['Rank'].astype('int')
Result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6736 entries, 0 to 6735
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         6736 non-null   int64  
 1   Bib          6385 non-null   float64
 2   FIS code     6736 non-null   int64  
 3   Athlete      5885 non-null   object 
 4   Year         6481 non-null   float64
 5   Nation       6140 non-null   object 
 6   Time         4366 non-null   object 
 7   Diff. Time   4555 non-null   object 
 8   FIS Points   4015 non-null   float64
 9   Event        6736 non-null   object 
 10  Event ID     6736 non-null   int64  
 11  Place        6736 non-null   object 
 12  Date         6736 non-null   object 
 13  Result Type  6736 non-null   object 
 14  Season       6736 non-null   int64  
dtypes: float64(3), int64(4), object(8)
memory usage: 789.5+ KB


In [112]:
def technique(Event):
    if "C/F" in Event:
        return "Skiathlon"
    elif "C" in Event:
        return 'Classic'
    elif "F" in Event:
        return 'Free'
    return 'Team'

def gender(Event):
    if "Women" in Event:
        return "Women"
    return "Men"

def team(Event):
    if "Team" in Event or "Rel" in Event:
        return "Team"
    return "Individual"

def sprintdistance(Event):
    if "SP" in Event or "sprint" in Event:
        return "Sprint"
    if "Overall" in Event:
        return "Overall"
    return "Distance"

def distance(Event):
    if "SP" in Event or "sprint" in Event:
        return "Sprint"
    elif "10" in Event:
        return "10"
    elif "15" in Event:
        return "15"
    elif "7.5/7.5" in Event:
        return "7.5/7.5"
    elif "15/15" in Event:
        return "15/15"
    elif "4x" in Event:
        return "Relay"
    elif "30" in Event:
        return "30"
    elif "50" in Event:
        return "50"
    elif "Overall" in Event:
        return "Overall"
    return "Other"

def points(ResultType):
    if "Overall" in ResultType:
        return 0
    elif "World Cup" in ResultType:
        return 1
    return 0

        
Result['Technique']=Result.apply(lambda x: technique(x['Event']),axis=1)
Result['Gender']=Result.apply(lambda x: gender(x['Event']),axis=1)
Result['Event individual/team']=Result.apply(lambda x: team(x['Event']),axis=1)
Result['Event sprint/distance']=Result.apply(lambda x: sprintdistance(x['Event']),axis=1)
Result['Distance']=Result.apply(lambda x: distance(x['Event']),axis=1)
Result['Points']=Result.apply(lambda x: points(x['Result Type']+x['Event']),axis=1)

Result
#Result.loc[(Result['FIS code']=='3185705') & (Result['Rank']<=3)]

,Rank,Bib,FIS code,Athlete,Year,Nation,Time,Diff. Time,FIS Points,Event,...,Place,Date,Result Type,Season,Technique,Gender,Event individual/team,Event sprint/distance,Distance,Points
0,1,31.0,3295157,SCARDONI Lucia,1991.0,ITA,3:03.01,3:03.01,0.00,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
1,2,12.0,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,3:04.16,+1.15,7.54,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
2,3,23.0,3295193,LAURENT Greta,1992.0,ITA,3:05.15,+2.14,14.03,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
3,4,4.0,3535304,CALDWELL Sophie,1990.0,USA,3:05.62,+2.61,17.11,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
4,5,27.0,3426112,STENSETH Ane Appelkvist,1995.0,NOR,3:05.67,+2.66,17.44,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6731,47,39.0,3530772,LUSTGARTEN Benjamin,1992.0,USA,2:21:25.4,+13:20.1,145.75,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1
6732,48,52.0,3181007,VUORELA Markus,1996.0,FIN,2:22:37.9,+14:32.6,158.96,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1
6733,49,56.0,3120063,SHANG Jincai,1993.0,CHN,2:23:13.9,+15:08.6,165.52,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1
6734,50,38.0,3530711,MARTIN Adam,1994.0,USA,2:23:15.4,+15:10.1,165.79,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1


In [113]:
athletes=Result[['FIS code','Athlete','Year','Nation']]
athletes=athletes.drop_duplicates(subset='FIS code', keep="first")
athletes=athletes.dropna()
athletes.loc[athletes['FIS code']==3295157]
athletes

,FIS code,Athlete,Year,Nation
0,3295157,SCARDONI Lucia,1991.0,ITA
1,3535320,MAUBET BJORNSEN Sadie,1989.0,USA
2,3295193,LAURENT Greta,1992.0,ITA
3,3535304,CALDWELL Sophie,1990.0,USA
4,3426112,STENSETH Ane Appelkvist,1995.0,NOR
...,...,...,...,...
6648,3425896,THEODORSEN Silje,1994.0,NOR
6669,3486003,VASILIEVA Lilia,1994.0,RUS
6673,3425772,BJOERNSGAARD Marthe,1993.0,NOR
6681,3665059,LUKONINA Ina,1994.0,BLR


In [114]:
#columns=['Athlete','Year','Nation',]
#Result.drop(columns, inplace=True, axis=1)
Result

,Rank,Bib,FIS code,Athlete,Year,Nation,Time,Diff. Time,FIS Points,Event,...,Place,Date,Result Type,Season,Technique,Gender,Event individual/team,Event sprint/distance,Distance,Points
0,1,31.0,3295157,SCARDONI Lucia,1991.0,ITA,3:03.01,3:03.01,0.00,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
1,2,12.0,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,3:04.16,+1.15,7.54,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
2,3,23.0,3295193,LAURENT Greta,1992.0,ITA,3:05.15,+2.14,14.03,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
3,4,4.0,3535304,CALDWELL Sophie,1990.0,USA,3:05.62,+2.61,17.11,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
4,5,27.0,3426112,STENSETH Ane Appelkvist,1995.0,NOR,3:05.67,+2.66,17.44,Women's SP C Qual,...,Ruka (FIN),"Nov 29, 2019",Sprint Qualification,2020,Classic,Women,Individual,Sprint,Sprint,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6731,47,39.0,3530772,LUSTGARTEN Benjamin,1992.0,USA,2:21:25.4,+13:20.1,145.75,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1
6732,48,52.0,3181007,VUORELA Markus,1996.0,FIN,2:22:37.9,+14:32.6,158.96,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1
6733,49,56.0,3120063,SHANG Jincai,1993.0,CHN,2:23:13.9,+15:08.6,165.52,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1
6734,50,38.0,3530711,MARTIN Adam,1994.0,USA,2:23:15.4,+15:10.1,165.79,Men's 50 km C Mst,...,Oslo (NOR),"Mar 08, 2020",World Cup,2020,Classic,Men,Individual,Distance,50,1


Left joinar på Wins på Athlete tabellen. 

Räknar endast när Rank=1 och Points=1 vilket betyder att det inte är ett kval eller overall slutresultat.

In [115]:
KPI = 'Podiums'
GroupBy = 'Points'
Definition = (Result['Rank'] <= 3) & (Result['Points']==1)

if KPI in athletes:
    athletes.drop(KPI, inplace=True, axis=1)

tmp_kpi=Result.loc[Definition]
tmp_kpi=tmp_kpi.groupby('FIS code')[GroupBy].value_counts()

athletes = pd.merge(athletes, tmp_kpi,how='left', on='FIS code')
athletes.rename(columns={GroupBy:KPI}, inplace=True)
athletes[KPI] = athletes[KPI].fillna(0)

athletes

,FIS code,Athlete,Year,Nation,Podiums
0,3295157,SCARDONI Lucia,1991.0,ITA,0.0
1,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,2.0
2,3295193,LAURENT Greta,1992.0,ITA,0.0
3,3535304,CALDWELL Sophie,1990.0,USA,2.0
4,3426112,STENSETH Ane Appelkvist,1995.0,NOR,0.0
...,...,...,...,...,...
470,3425896,THEODORSEN Silje,1994.0,NOR,0.0
471,3486003,VASILIEVA Lilia,1994.0,RUS,0.0
472,3425772,BJOERNSGAARD Marthe,1993.0,NOR,0.0
473,3665059,LUKONINA Ina,1994.0,BLR,0.0


In [116]:
KPI = 'Wins'
GroupBy = 'Points'
Definition = (Result['Rank'] == 1) & (Result['Points']==1)

if KPI in athletes:
    athletes.drop(KPI, inplace=True, axis=1)

tmp_kpi=Result.loc[Definition]
tmp_kpi=tmp_kpi.groupby('FIS code')[GroupBy].value_counts()

athletes = pd.merge(athletes, tmp_kpi,how='left', on='FIS code')
athletes.rename(columns={GroupBy:KPI}, inplace=True)
athletes[KPI] = athletes[KPI].fillna(0)

athletes

,FIS code,Athlete,Year,Nation,Podiums,Wins
0,3295157,SCARDONI Lucia,1991.0,ITA,0.0,0.0
1,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,2.0,0.0
2,3295193,LAURENT Greta,1992.0,ITA,0.0,0.0
3,3535304,CALDWELL Sophie,1990.0,USA,2.0,0.0
4,3426112,STENSETH Ane Appelkvist,1995.0,NOR,0.0,0.0
...,...,...,...,...,...,...
470,3425896,THEODORSEN Silje,1994.0,NOR,0.0,0.0
471,3486003,VASILIEVA Lilia,1994.0,RUS,0.0,0.0
472,3425772,BJOERNSGAARD Marthe,1993.0,NOR,0.0,0.0
473,3665059,LUKONINA Ina,1994.0,BLR,0.0,0.0


In [117]:
KPI = 'Top 10'
GroupBy = 'Points'
Definition = (Result['Rank'] <= 10) & (Result['Points']==1)

if KPI in athletes:
    athletes.drop(KPI, inplace=True, axis=1)

tmp_kpi=Result.loc[Definition]
tmp_kpi=tmp_kpi.groupby('FIS code')[GroupBy].value_counts()

athletes = pd.merge(athletes, tmp_kpi,how='left', on='FIS code')
athletes.rename(columns={GroupBy:KPI}, inplace=True)
athletes[KPI] = athletes[KPI].fillna(0)

athletes

,FIS code,Athlete,Year,Nation,Podiums,Wins,Top 10
0,3295157,SCARDONI Lucia,1991.0,ITA,0.0,0.0,1.0
1,3535320,MAUBET BJORNSEN Sadie,1989.0,USA,2.0,0.0,12.0
2,3295193,LAURENT Greta,1992.0,ITA,0.0,0.0,1.0
3,3535304,CALDWELL Sophie,1990.0,USA,2.0,0.0,9.0
4,3426112,STENSETH Ane Appelkvist,1995.0,NOR,0.0,0.0,4.0
...,...,...,...,...,...,...,...
470,3425896,THEODORSEN Silje,1994.0,NOR,0.0,0.0,0.0
471,3486003,VASILIEVA Lilia,1994.0,RUS,0.0,0.0,0.0
472,3425772,BJOERNSGAARD Marthe,1993.0,NOR,0.0,0.0,0.0
473,3665059,LUKONINA Ina,1994.0,BLR,0.0,0.0,0.0


In [118]:
athletes.info()
Result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475 entries, 0 to 474
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FIS code  475 non-null    int64  
 1   Athlete   475 non-null    object 
 2   Year      475 non-null    float64
 3   Nation    475 non-null    object 
 4   Podiums   475 non-null    float64
 5   Wins      475 non-null    float64
 6   Top 10    475 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 29.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6736 entries, 0 to 6735
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Rank                   6736 non-null   int64  
 1   Bib                    6385 non-null   float64
 2   FIS code               6736 non-null   int64  
 3   Athlete                5885 non-null   object 
 4   Year                   6481 non-null   float64
 5   Nation

In [119]:
#Result_tmp=Result.loc[Result['Place']=='Ruka (FIN)']
Result_tmp=Result
Result_tmp['Date']=pd.to_datetime(Result_tmp['Date'])
Result_tmp

athletes_tmp = athletes[['FIS code','Athlete','Year','Nation']]
athletes_tmp

,FIS code,Athlete,Year,Nation
0,3295157,SCARDONI Lucia,1991.0,ITA
1,3535320,MAUBET BJORNSEN Sadie,1989.0,USA
2,3295193,LAURENT Greta,1992.0,ITA
3,3535304,CALDWELL Sophie,1990.0,USA
4,3426112,STENSETH Ane Appelkvist,1995.0,NOR
...,...,...,...,...
470,3425896,THEODORSEN Silje,1994.0,NOR
471,3486003,VASILIEVA Lilia,1994.0,RUS
472,3425772,BJOERNSGAARD Marthe,1993.0,NOR
473,3665059,LUKONINA Ina,1994.0,BLR


In [122]:

# df['Rank']=df['Rank'].astype('int')
# df['Bib']=df['Bib'].astype('int')
# df['FIS code']=df['FIS code'].astype('int')
# df['FIS Points']=df['FIS Points'].astype('float')
# df['Year']=df['Year'].astype('int')

athletes_tmp.to_csv('transform/athletes.csv',encoding='utf-8',header=False)

Result_tmp.to_csv('transform/results.csv',encoding='utf-8',header=False)
Result_tmp.to_csv('transform/elo_test.csv',encoding='utf-8')


In [121]:
Result_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6736 entries, 0 to 6735
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Rank                   6736 non-null   int64         
 1   Bib                    6385 non-null   float64       
 2   FIS code               6736 non-null   int64         
 3   Athlete                5885 non-null   object        
 4   Year                   6481 non-null   float64       
 5   Nation                 6140 non-null   object        
 6   Time                   4366 non-null   object        
 7   Diff. Time             4555 non-null   object        
 8   FIS Points             4015 non-null   float64       
 9   Event                  6736 non-null   object        
 10  Event ID               6736 non-null   int64         
 11  Place                  6736 non-null   object        
 12  Date                   6736 non-null   datetime64[ns]
 13  Res